# USGS Stream Gage Site Metadata Downloader  
AUTH: Nathan T. Stevens  
ORG: Pacific Northwest Seismic Network  
LICENSE: GNU GPLv3  
PURPOSE: This notebook details how to get USGS surface water gage metadata (site information) from the WaterWatch dataservice. 

In [21]:
# Import dependencies
import pandas as pd
from pathlib import Path
import os

In [22]:
# USER PARAMETER BLOCK
# Define states to query and where to save the output
# Human readable list of USA state abbreviations
states = ['WA','OR']
# Where to save the file
PWD = Path().cwd()
SAVEDIR = PWD/'USGS_Stream_Gage'
# Create save path if it does not already exist
os.makedirs(SAVEDIR, exist_ok=True)

# END OF USER PARAMETER BLOCK


In [23]:
# Compose request URL for gauge site metadata
# Convert state list to URL format
states = ','.join(states).lower()
# Make request URL
URL = f'https://waterwatch.usgs.gov/download/?gt=map&mt=real&st={states}&dt=site&ht=&fmt=csv&mk=1'

In [24]:
# Submit request
df = pd.read_csv(URL, index_col='id')

In [25]:
# Parse `flowinfo` and `name` columns
_FI_HDRS = set(['Discharge (cfs)', 'Stage (ft)', 'Stage (adj) (ft)', 'Date', 'Length of record (years)', 'Class', '% normal(median) (%)', '% normal(mean) (%)', 'Status'])
holder = dict(zip(_FI_HDRS, [[] for _e in _FI_HDRS]))
idx = []
for id, row in df.iterrows():
    # Skip repeat header lines for multi-state query
    if id == 'id':
        continue
    # Catch station ID
    idx.append(id)

    # parse flowinfo
    fiparts = row.flowinfo.split(';')
    fi_claimed = set()
    for _e in fiparts:
        if 'Date' in _e:
            _k = _e[:4]
            _v = pd.Timestamp(_e[6:], tz='US/Pacific')
            holder[_k].append(_v)
            fi_claimed.add(_k)
            continue
        
        try:
            _k, _v = _e.split(':')
        except:
            breakpoint()
        _v = _v.strip()
        if _k in ['Class', 'Status']:
            pass
        else:
            _vp = _v.split(' ')
            try:
                _v = float(_vp[0])
            except:
                breakpoint()
            if _vp[1] != '%':
                _k += f' ({_vp[-1]})'
        holder[_k].append(_v)
        fi_claimed.add(_k)
    for _k, _v in holder.items():
        if len(_v) < len(idx):
            holder[_k].append(None)

dfp = pd.DataFrame(holder, index=idx)
df_parsed = df.join(dfp, how='left')
df_parsed.drop(columns=['flowinfo'], inplace=True)

In [26]:
# Display parsed site information & save to disk
display(df_parsed)
df_parsed.to_csv(SAVEDIR/'usgs_gauge_site_metadata.csv')

,name,lat,lng,class,url,huc_cd,Stage (ft),Discharge (cfs),Class,Length of record (years),Date,% normal(median) (%),% normal(mean) (%),Status,Stage (adj) (ft)
id,,,,,,,,,,,,,,,
10352500,"USGS 10352500 MCDERMITT CK NR MCDERMITT, NV",41.96655720,-117.83181200,4,https://waterdata.usgs.gov/monitoring-location...,16040201,2.23,3.77,10-24,74.0,2025-12-15 12:00:00-08:00,49.93,28.54,None,4547.23
10387110,USGS 10387110 CHEWAUCAN RIVER AT MOUTH NEAR VA...,42.52208056,-120.24945000,0,https://waterdata.usgs.gov/monitoring-location...,171200060506,8.52,NaN,Not-ranked,NaN,2025-12-15 12:00:00-08:00,NaN,NaN,None,NaN
10387150,"USGS 10387150 LAKE ABERT NEAR VALLEY FALLS, OR",42.60350000,-120.18730560,0,https://waterdata.usgs.gov/monitoring-location...,17120006,4253.30,NaN,Not-ranked,NaN,2025-12-15 12:45:00-08:00,NaN,NaN,None,4253.30
10396000,USGS 10396000 DONNER UND BLITZEN RIVER NR FREN...,42.79083330,-118.86750000,5,https://waterdata.usgs.gov/monitoring-location...,17120003,1.99,50.50,25-75,94.0,2025-12-15 12:00:00-08:00,120.24,90.19,None,4262.32
11491450,"USGS 11491450 IRVING CREEK NEAR LENZ, OR",42.95166667,-121.45905560,0,https://waterdata.usgs.gov/monitoring-location...,18010201,19.71,0.92,Not-ranked,NaN,2025-12-15 12:30:00-08:00,NaN,NaN,None,4636.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14244180,USGS 14244180 COWLITZ RIVER NEAR 1ST AVE NW AT...,46.14722008,-122.91605530,0,https://waterdata.usgs.gov/monitoring-location...,NaN,10.79,NaN,Not-ranked,NaN,2025-09-08 12:15:00-07:00,NaN,NaN,None,10.79
14246900,"USGS 14246900 COLUMBIA RIVER AT PORT WESTWARD,...",46.18122136,-123.18345390,5,https://waterdata.usgs.gov/monitoring-location...,17080003,8.24,211000.00,25-75,34.0,2025-12-15 12:40:00-08:00,92.14,82.78,None,7.24
14246900,"USGS 14246900 COLUMBIA RIVER AT PORT WESTWARD,...",46.18122136,-123.18345390,5,https://waterdata.usgs.gov/monitoring-location...,17080003,8.24,211000.00,25-75,34.0,2025-12-15 12:40:00-08:00,92.14,82.78,None,7.24
